# Initialisation de Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Exemples avec les formats de stockage") \
    .getOrCreate()
sc = spark.sparkContext

23/10/29 14:31:34 WARN Utils: Your hostname, sal9000 resolves to a loopback address: 127.0.1.1; using 192.168.1.201 instead (on interface eno2)
23/10/29 14:31:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/29 14:31:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Chargement des données

## Chargement des prénoms

In [8]:
from pyspark.sql import Row

# Charge un fichier texte et convertit les lignes en "Row".
lines = sc.textFile("prenoms_sample.txt")
prenoms_as_rdd = lines.map(lambda l: l.split(";"))\
    .map(lambda p: Row(\
        sexe=int(p[0]),\
        prenom=p[1],\
        annee=int(p[2]),\
        dep=p[3],\
        effectif=int(p[4])))

# Infère le schéma et enregistre le DataFrame comme une table.
prenoms = spark.createDataFrame(prenoms_as_rdd)
prenoms.createOrReplaceTempView("prenoms")
prenoms.show()

+----+--------------+-----+---+--------+
|sexe|        prenom|annee|dep|effectif|
+----+--------------+-----+---+--------+
|   1|_PRENOMS_RARES| 1909| 44|       7|
|   1|_PRENOMS_RARES| 1925| 64|      19|
|   1|_PRENOMS_RARES| 1926| 44|      21|
|   1|_PRENOMS_RARES| 1962| 66|      15|
|   1|_PRENOMS_RARES| 1997| 40|      15|
|   1|_PRENOMS_RARES| 2000| 46|      13|
|   1|_PRENOMS_RARES| 2016|973|    1463|
|   1|      ABDALLAH| 2007| 76|       3|
|   1|     ABDELAZIZ| 1978| 75|       3|
|   1|    ABDELHAFID| 1964| 42|       4|
|   1|    ABDELKADER| 2002| 78|       4|
|   1|    ABDELMALEK| 1982| 69|       4|
|   1|    ABDELMALIK| 1962| 75|       3|
|   1|         ABDON| 1920|971|       5|
|   1|     ABDOULLAH| 2016| 67|       3|
|   1|          ABEL| 1911| 71|       6|
|   1|          ABEL| 1921| 04|       7|
|   1|          ABEL| 2019| 37|      15|
|   1|     ABOUBACAR| 1994| 13|       3|
|   1|       ABRAHAM| 2017| 75|       7|
+----+--------------+-----+---+--------+
only showing top

## Chargement des départements

In [6]:
lines = sc.textFile("dpts.txt")
depts_as_rdd = lines\
    .filter(lambda l: "dep" not in l and "2A" not in l and "2B" not in l)\
    .map(lambda l: l.split(","))\
    .map(lambda p: Row(\
        dep=int(p[0]),\
        reg=int(p[1]),\
        cheflieu=p[2],\
        tncc=p[3],\
        ncc=p[4],\
        nccenr=p[6],\
        libelle=p[6]))

depts = spark.createDataFrame(depts_as_rdd)
depts.createOrReplaceTempView("depts")
depts.show()

+---+---+--------+----+--------------------+--------------------+--------------------+
|dep|reg|cheflieu|tncc|                 ncc|              nccenr|             libelle|
+---+---+--------+----+--------------------+--------------------+--------------------+
|  1| 84|   01053|   5|                 AIN|                 Ain|                 Ain|
|  2| 32|   02408|   5|               AISNE|               Aisne|               Aisne|
|  3| 84|   03190|   5|              ALLIER|              Allier|              Allier|
|  4| 93|   04070|   4|ALPES DE HAUTE PR...|Alpes-de-Haute-Pr...|Alpes-de-Haute-Pr...|
|  5| 93|   05061|   4|        HAUTES ALPES|        Hautes-Alpes|        Hautes-Alpes|
|  6| 93|   06088|   4|     ALPES MARITIMES|     Alpes-Maritimes|     Alpes-Maritimes|
|  7| 84|   07186|   5|             ARDECHE|             Ardèche|             Ardèche|
|  8| 44|   08105|   4|            ARDENNES|            Ardennes|            Ardennes|
|  9| 76|   09122|   5|              ARIEGE

# Sauvegarder les données au format parquet

## Les prénoms partitionnés par départements et années et compressés (Snappy)

In [9]:
prenoms.write\
    .partitionBy('dep', 'annee')\
        .format('parquet')\
            .save('prenomsParDeptsEtAnnees.parquet')

## Les prénoms partitionnés par départements et années et compressés (gzip)

In [10]:
prenoms.write\
    .partitionBy('dep', 'annee')\
        .option("compression", "gzip")\
            .format('parquet')\
                .save("prenomsParDeptsEtAnnees.gzip.parquet")

## Les départements

In [11]:
depts.write\
    .format("parquet")\
        .save("depts.parquet")